# Twitter web scrapper

We will utilize Selenium and undetected_chromedriver to perform this task.

In [1]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import random
import numpy as np
import time
import csv
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

TWITTER_USER = os.getenv("TWITTER_USER")
TWITTER_PASS = os.getenv("TWITTER_PASS")

In [3]:

# --- Setup undetected Chrome ---
options = uc.ChromeOptions()
# options.add_argument("--headless")  # Optional for silent mode
driver = uc.Chrome(options=options,version_main=137)

driver.maximize_window()


In [4]:
def log_in(driver):
    # --- Login to Twitter ---
    driver.get("https://x.com/login")
    time.sleep(5)

    username_input = driver.find_element(By.NAME, "text")
    username_input.send_keys(TWITTER_USER)
    username_input.send_keys(Keys.RETURN)
    time.sleep(3)

    # Optional extra check
    try:
        confirm_user = driver.find_element(By.NAME, "text")
        confirm_user.send_keys(TWITTER_USER)
        confirm_user.send_keys(Keys.RETURN)
        time.sleep(3)
    except:
        pass

    password_input = driver.find_element(By.NAME, "password")
    password_input.send_keys(TWITTER_PASS)
    password_input.send_keys(Keys.RETURN)
    time.sleep(5)

In [7]:
log_in(driver)

In [17]:
from tqdm.notebook import tqdm
from IPython.display import clear_output

In [ ]:
wait_seconds = list(np.arange(5,10))
wait_short = list(np.arange(2,5))

def search_faitel_tweets(since_,until_):
    query = f"from:DavidFaitelson_ since:{since_} until:{until_}"
    search_url = f"https://x.com/search?q={query}&src=typed_query&f=live"
    driver.get(search_url)
    time.sleep(random.choice(wait_seconds))

    for _ in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(random.choice(wait_short))

    tweets = driver.find_elements(By.XPATH, '//article[@data-testid="tweet"]')
    results = []

    for tweet in tweets:
        try:
            content = tweet.text
            link = tweet.find_element(By.XPATH, './/a[contains(@href, "/status/")]')
            tweet_url = link.get_attribute('href')
            tweet_id = tweet_url.split("/")[-1]
            results.append({"tweet_id": tweet_id, "text": content})
        except Exception as e:
            print("Skipped a tweet due to error:", e)

    filename = "david_faitelson_tweets.csv"
    with open(filename, "a", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=["tweet_id", "text"])
        writer.writerows(results)

    return len(results)



In [20]:
import sys
import calendar

In [ ]:
all_tweets = 0
# for y in np.arange(2019,2022):
for y in [2022]:
    for m in np.arange(1,13):
        iter_c = 0
        while True:
            
            m_text = ("0" if m<10 else "")+str(m)
            s = f"{y}-{m_text}-01"
            u = f"{y}-{m_text}-10"
            c = search_faitel_tweets(s,u)
            all_tweets += c
            iter_c += c
            sys.stdout.write(f"\n✅ Done! Saved {c} since: {s} until: {u}. Total: {all_tweets}")
            sys.stdout.flush()

            s = f"{y}-{m_text}-11"
            u = f"{y}-{m_text}-20"
            c = search_faitel_tweets(s,u)
            all_tweets += c
            iter_c += c
            sys.stdout.write(f"\n✅ Done! Saved {c} since: {s} until: {u}. Total: {all_tweets}")
            sys.stdout.flush()

            s = f"{y}-{m_text}-21"
            last_day = calendar.monthrange(y, m)[1]
            u = f"{y}-{m_text}-{last_day}"
            c = search_faitel_tweets(s,u)
            all_tweets += c
            iter_c += c
            sys.stdout.write(f"\n✅ Done! Saved {c} since: {s} until: {u}. Total: {all_tweets}")
            sys.stdout.flush()

            if iter_c==0:
                sys.stdout.write(f"\n Retrying {m} {y}")
                sys.stdout.flush()
                time.sleep(5*60)
            else:
                break


✅ Done! Saved 2 since: 2022-01-01 until: 2022-01-10. Total: 2
✅ Done! Saved 24 since: 2022-01-11 until: 2022-01-20. Total: 26
✅ Done! Saved 21 since: 2022-01-21 until: 2022-01-31. Total: 47
✅ Done! Saved 18 since: 2022-02-01 until: 2022-02-10. Total: 65
✅ Done! Saved 28 since: 2022-02-11 until: 2022-02-20. Total: 93
✅ Done! Saved 23 since: 2022-02-21 until: 2022-02-28. Total: 116
✅ Done! Saved 18 since: 2022-03-01 until: 2022-03-10. Total: 134
✅ Done! Saved 20 since: 2022-03-11 until: 2022-03-20. Total: 154
✅ Done! Saved 23 since: 2022-03-21 until: 2022-03-31. Total: 177
✅ Done! Saved 25 since: 2022-04-01 until: 2022-04-10. Total: 202
✅ Done! Saved 26 since: 2022-04-11 until: 2022-04-20. Total: 228
✅ Done! Saved 0 since: 2022-04-21 until: 2022-04-30. Total: 228
✅ Done! Saved 0 since: 2022-05-01 until: 2022-05-10. Total: 228
✅ Done! Saved 0 since: 2022-05-11 until: 2022-05-20. Total: 228
✅ Done! Saved 0 since: 2022-05-21 until: 2022-05-31. Total: 228
 Retrying 5 2022
✅ Done! Saved 0 si

KeyboardInterrupt: 

In [11]:
driver.quit()

In [ ]:
def read_replies(driver,tweet_id):
    # Open the tweet
    url = f"https://x.com/DavidFaitelson_/status/{tweet_id}"
    driver.get(url)

    # Wait for replies to load
    time.sleep(7)

    # Scroll 
    for _ in range(1):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(random.choice(wait_short))

    tws = driver.find_elements(By.XPATH, '//article[@data-testid="tweet"]')
    results = []

    for tweet in tws:
        try:
            content = tweet.text
            link = tweet.find_element(By.XPATH, './/a[contains(@href, "/status/")]')
            tweet_url = link.get_attribute('href')
            reply_id = tweet_url.split("/")[-1]
            
            results.append({"tweet_id":tweet_id, "reply_id": reply_id, "text": content})
        except Exception as e:
            print("Skipped a tweet due to error:", e)


    with open('replies.csv', mode='a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["tweet_id","reply_id", "text"])
        writer.writerows(results)

    return len(results)
            

In [6]:
import pandas as pd

In [17]:
tweets =pd.read_csv("david_faitelson_tweets.csv",dtype={"tweet_id":"str"})
print(tweets.shape)
tweets.head()

(4205, 2)


,tweet_id,text
0,1225506069386055680,"David Faitelson\n@DavidFaitelson_\n·\nFeb 6, 2..."
1,1225501059793158145,"David Faitelson\n@DavidFaitelson_\n·\nFeb 6, 2..."
2,1225499608165556226,"David Faitelson\n@DavidFaitelson_\n·\nFeb 6, 2..."
3,1225490334878175233,"David Faitelson\n@DavidFaitelson_\n·\nFeb 6, 2..."
4,1225489331420356609,"David Faitelson\n@DavidFaitelson_\n·\nFeb 6, 2..."


In [18]:
tweets.iloc[-1].text

'David Faitelson\n@DavidFaitelson_\n·\nJun 21, 2022\nReplying to \n@jsanchez_w\n and \n@TUDNMEX\nAbrazo, Jorge, mi más sentido pésame…\n1\n57'

In [ ]:
rets = pd.read_csv("replies.csv",dtype={"tweet_id":"str","reply_id":"str"})
f = pd.read_csv("david_faitelson_tweets.csv",dtype={"tweet_id":"str"})
last_tweet_id = f[f.tweet_id==rets.iloc[-1].tweet_id].index[0]
last_tweet_id

In [ ]:
all_replies = 0
faitel_tweets = tweets.iloc[last_tweet_id+1:]
for i,t in tqdm(faitel_tweets.iterrows(),total=len(faitel_tweets),desc="Processing"):
    c = read_replies(driver,t.tweet_id)
    all_replies+=c
    sys.stdout.write(f'\r✅ Done! Saved {c} replies. Total: {all_replies}')
    sys.stdout.flush()

In [24]:
driver.close()

In [11]:
replies = pd.read_csv("replies.csv",dtype={"tweet_id":'str','reply_id':"str"})
faitel_tweets = pd.read_csv("david_faitelson_tweets.csv",dtype={"tweet_id":'str'})
faitel_tweets.drop_duplicates(subset="tweet_id",keep="first",inplace=True)
print(f"Faitel tweets {faitel_tweets.shape[0]}",f"Replies {replies.shape[0]}")

Faitel tweets 3974 Replies 85134


In [12]:
from datetime import datetime

def is_valid_date(text):
    try:
        datetime.strptime(text, "%b %d, %Y")
        return True
    except ValueError:
        try: 
            cleaned = text.replace('·', '').strip()
            datetime.strptime(cleaned, "%I:%M %p %b %d, %Y")
            return True
        except ValueError:
            return False

def get_user(text):
    rows = text.split("\n")
    for r in rows:
        if "@"==r.strip()[:1]:
            return r.strip()
    return None

def format_text(text):
    invalid_text = [".","·","Translate post"]
    rows = text.split("\n")
    res = []
    for t in rows[2:]:
        t = t.strip()
        if (is_valid_date(t)==False) & (t.isnumeric()==False) & ((t in invalid_text)==False):
            res.append(t)
    return "\n".join(res)

In [10]:
replies['user'] = replies.text.map(get_user)
replies['tweet_text'] = replies.text.map(format_text)

In [13]:
replies_by_tweet = replies[replies.tweet_id!=replies.reply_id].groupby("tweet_id").size()

In [14]:
replies_by_tweet

tweet_id
1004506165126938624    27
1004509850804424704    35
1004557419677126664    20
1004582036303241216    32
1004583057138036736    25
                       ..
999772267016994817      7
999785561547128834     35
999788684437217281     29
999790742473093120     26
999848407433793538     15
Length: 3597, dtype: int64

In [15]:
missing_tweets = faitel_tweets[~faitel_tweets.tweet_id.isin(replies_by_tweet.index)]
missing_tweets.shape

(377, 2)

In [22]:
all_replies = 0

for i,t in tqdm(missing_tweets.iterrows(),total=len(missing_tweets),desc="Processing"):
    tries =1
    while True:
        c = read_replies(driver,t.tweet_id)
        if (c==0) and (tries==1):
            tries+=1
            sys.stdout.write(f'\r✅ Retrying tweet {t.tweet_id}')
            sys.stdout.flush()
            time.sleep(315)
        else:
            break

    all_replies+=c
    sys.stdout.write(f'\r✅ Done! Saved {c} replies. Total: {all_replies}')
    sys.stdout.flush()

Processing:   0%|          | 0/377 [00:00<?, ?it/s]

✅ Done! Saved 2 replies. Total: 68131

In [23]:
driver.close()